In [ ]:
# Replication of Paper by Loughran and McDonald 
# Group members: Lilian Yu (jiajieyu), Ziyi Shen (ziyis), Tristan Zhang (jinyang3)


In [1]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
# import project_helper

from tqdm import tqdm

In [2]:
# project_helper

import matplotlib.pyplot as plt
import requests

from ratelimit import limits, sleep_and_retry


class SecAPI(object):
    SEC_CALL_LIMIT = {'calls': 10, 'seconds': 1}

    @staticmethod
    @sleep_and_retry
    @limits(calls=SEC_CALL_LIMIT['calls'] / 2, period=SEC_CALL_LIMIT['seconds'])
    def _call_sec(url):
        return requests.get(url)

    def get(self, url):
        return self._call_sec(url).text


def print_ten_k_data(ten_k_data, fields, field_length_limit=50):
    indentation = '  '

    print('[')
    for ten_k in ten_k_data:
        print_statement = '{}{{'.format(indentation)
        for field in fields:
            value = str(ten_k[field])

            # Show return lines in output
            if isinstance(value, str):
                value_str = '\'{}\''.format(value.replace('\n', '\\n'))
            else:
                value_str = str(value)

            # Cut off the string if it gets too long
            if len(value_str) > field_length_limit:
                value_str = value_str[:field_length_limit] + '...'

            print_statement += '\n{}{}: {}'.format(indentation * 2, field, value_str)

        print_statement += '},'
        print(print_statement)
    print(']')


def plot_similarities(similarities_list, dates, title, labels):
    assert len(similarities_list) == len(labels)

    plt.figure(1, figsize=(10, 7))
    for similarities, label in zip(similarities_list, labels):
        plt.title(title)
        plt.plot(dates, similarities, label=label)
        plt.legend()
        plt.xticks(rotation=90)

    plt.show()

In [3]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/angelina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/angelina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/angelina/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
#cik_lookup = {
  #  'AMZN': '0001018724',
 #   'BMY': '0000014272',   
  #  'CNP': '0001130310',
  #  'CVX': '0000093410',
 #   'FL': '0000850209',
 #   'FRT': '0000034903',
 #   'HON': '0000773840'}
#cik_lookup = {
  # 'AMZN': '0001018724',
 #   'BMY': '0000014272'}

In [6]:
#import tick and cik
#from google.colab import files
#uploaded = files.upload()

#import io
df2 = pd.read_csv('sp500_w_addl_id_with_cik.csv')
# Dataset is now stored in a Pandas Dataframe

/var/folders/d4/k_xt3lys2znb_23f5xrq4y600000gn/T/ipykernel_17704/2012727981.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('sp500_w_addl_id_with_cik.csv')


In [7]:
df = df2[['date','ticker', 'cik']]
df=df.loc[df['date']>='2017-01-01']
df

,date,ticker,cik
6013,2017/1/31,MTB,36270.0
6014,2017/1/31,AN,350698.0
6015,2017/1/31,FRT,34903.0
6016,2017/1/31,ITW,49826.0
6017,2017/1/31,CPB,16732.0
...,...,...,...
36053,2021/12/31,IRM,1020569.0
36054,2021/12/31,AZO,866787.0
36055,2021/12/31,MDT,1613103.0
36056,2021/12/31,HPQ,47217.0


In [8]:
tickers = df.groupby('date')['ticker'].apply(list)
ciks = df.groupby('date')['cik'].apply(list)
tickers,ciks
#sp500=df.groupby('ticker')['date'].apply(list)
#sp500

(date
 2017/1/31     [MTB, AN, FRT, ITW, CPB, ABT, PLD, JCI, BBBY, ...
 2017/10/31    [L, BAX, DUK, DIS, XRX, PNW, APD, DOV, VNO, F,...
 2017/11/30    [GPC, AMP, NTAP, WDC, PEP, MOS, FLS, MAR, PPG,...
 2017/12/29    [SYK, CDNS, HBAN, AET, MAC, WY, MDLZ, COG, HSY...
 2017/2/28     [DISCA, UTX, ROK, AIZ, PVH, LLTC, SJM, FISV, T...
 2017/3/31     [T, ES, NEE, SEE, HIG, UHS, KLAC, CSRA, WYN, L...
 2017/4/28     [AMD, SEE, FMC, NTRS, EXPE, APA, EQT, HCN, DE,...
 2017/5/31     [KEY, TMO, STZ, LYB, AMP, GM, NVDA, ETR, TAP, ...
 2017/6/30     [LLL, BK, FBHS, TROW, FRT, EXR, BWA, HOLX, YUM...
 2017/7/31     [HCA, PVH, ROST, BA, CPB, BBT, UNP, CBOE, UTX,...
 2017/8/31     [CHD, FRT, AIG, MHK, INTU, EVHC, WM, CHTR, TRO...
 2017/9/29     [EVHC, VZ, AIG, GIS, HCN, ORCL, MHK, INTU, HAS...
 2018/1/31     [AMP, CHRW, GWW, HPQ, LNT, ETN, CFG, HCA, PKI,...
 2018/10/31    [D, FLS, CMS, ADM, VAR, EFX, GM, CMA, NOC, ISR...
 2018/11/30    [BDX, GD, SBAC, PEG, HCA, EFX, PPG, RHT, COST,...
 2018/12/31    [BDX

In [9]:
#tickers.keys()
#
#df.loc[df['cik'].isna()] AA and DPS have no cik listed
df = df[df.cik.notnull()]
#df['cik'].astype(int)
#cik_lookup=dict(zip(df.ticker[:5], df['cik'][:5].astype(int)))
#cik_lookup                                                          ##pick first 5

In [10]:
look= df.groupby('ticker')['cik'].apply(list)
look=look.to_dict()
#lookup=dict(zip(look.ticker, look['ticker'][0].astype(int)))
#lookup
cik_lookupss={}
for ticker, ciks in look.items():
    cik_lookupss[ticker]=int(ciks[0])
##cik_lookupss.keys()     ##656 companies

In [11]:
dates = df2['date'].unique()

In [12]:
ALL_ticker=list(cik_lookupss.keys())

In [13]:
########
num = 401
cik_lookup= { key: cik_lookupss[key] for key in ALL_ticker[num:(num+50)] }   #first 50 comps
#cik_lookup

In [14]:
######10K

sec_api = SecAPI()

from bs4 import BeautifulSoup

HEADER = {'Host': 'www.sec.gov', 'Connection': 'close',
         'Accept': 'application/json, text/javascript, */*; q=0.01', 'X-Requested-With': 'XMLHttpRequest',
         'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36',
         }
def get_sec_data(cik, doc_type, start=0, count=60):
  rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
      '&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom' \
      .format(cik, doc_type, start, count)
  sec_data = requests.get(rss_url,headers=HEADER)
  feed = BeautifulSoup(sec_data.content, 'html')
  entries = [
      (
          entry.content.find('filing-href').getText(),
          entry.content.find('filing-type').getText(),
          entry.content.find('filing-date').getText())
      for entry in feed.find_all('entry')]
  return entries

example_ticker = ALL_ticker[num]
sec_data = {}   #dict 10K
for ticker, cik in cik_lookup.items():
    sec_data[ticker] = get_sec_data(cik, '10-K')[0:5] ##
pprint.pprint(sec_data[example_ticker])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


[('https://www.sec.gov/Archives/edgar/data/1120193/000112019322000007/0001120193-22-000007-index.htm',
  '10-K',
  '2022-02-23'),
 ('https://www.sec.gov/Archives/edgar/data/1120193/000112019321000011/0001120193-21-000011-index.htm',
  '10-K',
  '2021-02-23'),
 ('https://www.sec.gov/Archives/edgar/data/1120193/000112019320000004/0001120193-20-000004-index.htm',
  '10-K',
  '2020-02-25'),
 ('https://www.sec.gov/Archives/edgar/data/1120193/000112019319000002/0001120193-19-000002-index.htm',
  '10-K',
  '2019-02-22'),
 ('https://www.sec.gov/Archives/edgar/data/1120193/000112019318000003/0001120193-18-000003-index.htm',
  '10-K',
  '2018-02-28')]


In [15]:
#####10Q

sec_api = SecAPI()

from bs4 import BeautifulSoup

HEADER = {'Host': 'www.sec.gov', 'Connection': 'close',
         'Accept': 'application/json, text/javascript, */*; q=0.01', 'X-Requested-With': 'XMLHttpRequest',
         'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36',
         }
def get_sec_data(cik, doc_type, start=0, count=60):
  rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
      '&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom' \
      .format(cik, doc_type, start, count)
  sec_data = requests.get(rss_url,headers=HEADER)
  feed = BeautifulSoup(sec_data.content, 'html')
  entries = [
      (
          entry.content.find('filing-href').getText(),
          entry.content.find('filing-type').getText(),
          entry.content.find('filing-date').getText())
      for entry in feed.find_all('entry')]
  return entries

example_ticker = ALL_ticker[num]
sec_data_10Q = {}     #dict 10Q
for ticker, cik in cik_lookup.items():
    sec_data_10Q[ticker] = get_sec_data(cik, '10-Q')[2:18]
pprint.pprint(sec_data_10Q[example_ticker])

[('https://www.sec.gov/Archives/edgar/data/1120193/000112019321000027/0001120193-21-000027-index.htm',
  '10-Q',
  '2021-11-04'),
 ('https://www.sec.gov/Archives/edgar/data/1120193/000112019321000022/0001120193-21-000022-index.htm',
  '10-Q',
  '2021-08-04'),
 ('https://www.sec.gov/Archives/edgar/data/1120193/000112019321000016/0001120193-21-000016-index.htm',
  '10-Q',
  '2021-05-05'),
 ('https://www.sec.gov/Archives/edgar/data/1120193/000112019320000021/0001120193-20-000021-index.htm',
  '10-Q',
  '2020-11-04'),
 ('https://www.sec.gov/Archives/edgar/data/1120193/000112019320000015/0001120193-20-000015-index.htm',
  '10-Q',
  '2020-08-05'),
 ('https://www.sec.gov/Archives/edgar/data/1120193/000112019320000007/0001120193-20-000007-index.htm',
  '10-Q',
  '2020-05-06'),
 ('https://www.sec.gov/Archives/edgar/data/1120193/000112019319000015/0001120193-19-000015-index.htm',
  '10-Q',
  '2019-11-06'),
 ('https://www.sec.gov/Archives/edgar/data/1120193/000112019319000010/0001120193-19-000010

In [16]:
#periods=pd.DataFrame()
#periods['End']=tickers.keys()
#periods['Start']=periods.End.shift(1)
#periods['Start'][0]='2017-01-01'
#periods.head()

In [17]:
#for ticker in cik_lookup.keys():
#  print(ticker)

In [18]:
for ticker in cik_lookup.keys():     ##10K  keep all in the time horizon 5 years
  for item in sec_data[ticker]:
    if (item[-1]<'2017-01-03') or (item[-1]>'2021-12-31'):
      sec_data[ticker].remove(item)

In [19]:
for ticker in cik_lookup.keys():   
  for item in sec_data_10Q[ticker]:     ##10Q  keep all in the time horizon
    if (item[-1]<'2017-01-03') or (item[-1]>'2021-12-31'):
      sec_data_10Q[ticker].remove(item)

In [20]:
#sec_data_10Q[ticker]  Now all 10Q files are in date range

In [21]:
####Take long time, could play around for a while


raw_fillings_by_ticker = {}     
for ticker, data in sec_data.items():   ###get 10K from dict
    try:
        raw_fillings_by_ticker[ticker] = {}   ###create empty 10K filing
        for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
            if (file_type == '10-K'):
                file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            
            
                raw_fillings_by_ticker[ticker][file_date] = requests.get(file_url,headers=HEADER).text
    except:
        print(ticker)
print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker[example_ticker].values()))[:1000]))

Example Document:

<SEC-DOCUMENT>0001120193-21-000011.txt : 20210223
<SEC-HEADER>0001120193-21-000011.hdr.sgml : 20210223
<ACCEPTANCE-DATETIME>20210223164938
ACCESSION NUMBER:		0001120193-21-000011
CONFORMED SUBMISSION TYPE:	10-K
PUBLIC DOCUMENT COUNT:		152
CONFORMED PERIOD OF REPORT:	20201231
FILED AS OF DATE:		20210223
DATE AS OF CHANGE:		20210223

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			NASDAQ, INC.
		CENTRAL INDEX KEY:			0001120193
		STANDARD INDUSTRIAL CLASSIFICATION:	SECURITY & COMMODITY BROKERS, DEALERS, EXCHANGES & SERVICES [6200]
		IRS NUMBER:				521165937
		STATE OF INCORPORATION:			DE
		FISCAL YEAR END:			1231

	FILING VALUES:
		FORM TYPE:		10-K
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	001-38855
		FILM NUMBER:		21666374

	BUSINESS ADDRESS:	
		STREET 1:		151 W. 42ND STREET
		CITY:			NEW YORK
		STATE:			NY
		ZIP:			10036
		BUSINESS PHONE:		1 212 401 8700

	MAIL ADDRESS:	
		STREET 1:		151 W. 42ND STREET
		CITY:			NEW YORK
		STATE:			NY
		ZIP:			10036

	FORMER COMPANY

In [22]:
import re
def get_documents(text):
    extracted_docs = []
    
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')   
    
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(text)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(text)]
    
    for doc_start_i, doc_end_i in zip(doc_start_is, doc_end_is):
            extracted_docs.append(text[doc_start_i:doc_end_i])
    
    return extracted_docs

filling_documents_by_ticker = {}
for ticker, raw_fillings in raw_fillings_by_ticker.items():
    try:
        filling_documents_by_ticker[ticker] = {}
        for file_date, filling in tqdm(raw_fillings.items(), desc='Getting Documents from {} Fillings'.format(ticker), unit='filling'):
            filling_documents_by_ticker[ticker][file_date] = get_documents(filling)
    except:
        print(ticker)
print('\n\n'.join([
    'Document {} Filed on {}:\n{}...'.format(doc_i, file_date, doc[:200])
    for file_date, docs in filling_documents_by_ticker[example_ticker].items()
    for doc_i, doc in enumerate(docs)][:3]))

Getting Documents from ODFL Fillings: 100%|██████████| 4/4 [00:00<00:00, 33.07filling/s]
Getting Documents from OGN Fillings: 0filling [00:00, ?filling/s]
Getting Documents from PHM Fillings: 100%|██████████| 4/4 [00:00<00:00,  7.07filling/s]

Document 0 Filed on 2021-02-23:

<TYPE>10-K
<SEQUENCE>1
<FILENAME>ndaq-20201231.htm
<DESCRIPTION>10-K
<TEXT>
<XBRL>
<?xml version="1.0" ?><!--XBRL Document Created with Wdesk from Workiva--><!--Copyright 2021 Workiva--><!--r:cd870a3...

Document 1 Filed on 2021-02-23:

<TYPE>EX-2.2
<SEQUENCE>2
<FILENAME>ndaq12312020ex-22.htm
<DESCRIPTION>EX-2.2
<TEXT>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head>
...

Document 2 Filed on 2021-02-23:

<TYPE>EX-2.3
<SEQUENCE>3
<FILENAME>ndaq12312020ex-23.htm
<DESCRIPTION>EX-2.3
<TEXT>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head>
...


In [23]:
def get_document_type(doc):
    
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    
    doc_type = type_pattern.findall(doc)[0][len('<TYPE>'):] 
    
    return doc_type.lower()

In [24]:
ten_ks_by_ticker = {}
for ticker, filling_documents in filling_documents_by_ticker.items():
    try:
        ten_ks_by_ticker[ticker] = []
        for file_date, documents in filling_documents.items():
            for document in documents:
                if get_document_type(document) == '10-k':
                    ten_ks_by_ticker[ticker].append({
                        'cik': cik_lookup[ticker],
                        'file': document,
                        'file_date': file_date})
    except:
        print(ticker)
print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['cik', 'file', 'file_date'])

[
  {
    cik: '1120193'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>ndaq-2020123...
    file_date: '2021-02-23'},
  {
    cik: '1120193'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>ndaq12312019...
    file_date: '2020-02-25'},
  {
    cik: '1120193'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>ndaq12312018...
    file_date: '2019-02-22'},
  {
    cik: '1120193'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>ndaq12312017...
    file_date: '2018-02-28'},
]


In [25]:
def remove_html_tags(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    return text
def clean_text(text):
    text = text.lower().strip()
    text = remove_html_tags(text)
    #text = re.sub("[^a-zA-Z]"," ",text).split(' ')  #this will take off the release date, do it later before 
    
    return text



In [26]:
####Take long time, could play around for a while  ~10min for 50 comps 10K

for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        for ten_k in tqdm(ten_ks, desc='Cleaning {} 10-Ks'.format(ticker), unit='10-K'):
            ten_k['file_clean'] = clean_text(ten_k['file'])
    except:
        print(ticker)
print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_clean'])

Cleaning ODFL 10-Ks: 100%|██████████| 4/4 [00:06<00:00,  1.58s/10-K]
Cleaning OGN 10-Ks: 010-K [00:00, ?10-K/s]
Cleaning PHM 10-Ks: 100%|██████████| 4/4 [00:11<00:00,  2.84s/10-K]

[
  {
    file_clean: '10-k\n1\nndaq-20201231.htm\n10-k\n\n\n\nndaq-2020...},
  {
    file_clean: '10-k\n1\nndaq1231201910-k.htm\n10-k\n\n\n\n\n\n\n...},
  {
    file_clean: '10-k\n1\nndaq1231201810-k.htm\n10-k\n\n\n\n\n\n\n...},
  {
    file_clean: '10-k\n1\nndaq1231201710-k.htm\n10-k\n\n\n\n\n\n\n...},
]


In [27]:
#print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_date'])
print_ten_k_data(ten_ks_by_ticker[example_ticker][0:1], ['file_clean'])

[
  {
    file_clean: '10-k\n1\nndaq-20201231.htm\n10-k\n\n\n\nndaq-2020...},
]


In [28]:
ten_ks_by_ticker[example_ticker][0]['file_date']  ##the 0th company's file date

'2021-02-23'

lemmatize all the data

In [29]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
def lemmatize_words(words):

    lemmatized_words = [WordNetLemmatizer().lemmatize(word, 'v') for word in words]
    
    return lemmatized_words
word_pattern = re.compile('\w+')
for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        for ten_k in tqdm(ten_ks, desc='Lemmatize {} 10-Ks'.format(ticker), unit='10-K'):
            ten_k['file_lemma'] = lemmatize_words(word_pattern.findall(ten_k['file_clean']))
    except:
        print(ticker)
print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_lemma'])

Lemmatize ODFL 10-Ks: 100%|██████████| 4/4 [00:00<00:00,  6.2110-K/s]
Lemmatize OGN 10-Ks: 010-K [00:00, ?10-K/s]
Lemmatize PHM 10-Ks: 100%|██████████| 4/4 [00:00<00:00,  4.5710-K/s]

[
  {
    file_lemma: '['10', 'k', '1', 'ndaq', '20201231', 'htm', '10',...},
  {
    file_lemma: '['10', 'k', '1', 'ndaq1231201910', 'k', 'htm', '1...},
  {
    file_lemma: '['10', 'k', '1', 'ndaq1231201810', 'k', 'htm', '1...},
  {
    file_lemma: '['10', 'k', '1', 'ndaq1231201710', 'k', 'htm', '1...},
]


Remove the stopwords.

In [30]:
from nltk.corpus import stopwords
lemma_english_stopwords = lemmatize_words(stopwords.words('english'))
for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        for ten_k in tqdm(ten_ks, desc='Remove Stop Words for {} 10-Ks'.format(ticker), unit='10-K'):
            ten_k['file_lemma'] = [word for word in ten_k['file_lemma'] if word not in lemma_english_stopwords]
    except:
        print(ticker)
print('Stop Words Removed')

Remove Stop Words for ODFL 10-Ks: 100%|██████████| 4/4 [00:00<00:00, 13.8010-K/s]
Remove Stop Words for OGN 10-Ks: 010-K [00:00, ?10-K/s]
Remove Stop Words for PHM 10-Ks: 100%|██████████| 4/4 [00:00<00:00,  9.9410-K/s]

Stop Words Removed


In [31]:
ten_k['file_lemma']

['10',
 'k',
 '1',
 'a201710',
 'k',
 'htm',
 '10',
 'k',
 '2017',
 'document',
 'unite',
 'statessecurities',
 'exchange',
 'commissionwashington',
 'c',
 '20549form',
 '10',
 'k',
 'x',
 'annual',
 'report',
 'pursuant',
 'section',
 '13',
 '15',
 'securities',
 'exchange',
 'act',
 '1934for',
 'fiscal',
 'year',
 'end',
 'december',
 '31',
 '2017',
 'transition',
 'report',
 'pursuant',
 'section',
 '13',
 '15',
 'securities',
 'exchange',
 'act',
 '1934commission',
 'file',
 'number',
 '1',
 '9804',
 '_______________________________________________________________________pultegroup',
 'inc',
 'exact',
 'name',
 'registrant',
 'specify',
 'charter',
 'michigan',
 '38',
 '2766606',
 'state',
 'jurisdiction',
 'ofincorporation',
 'organization',
 'r',
 'employeridentification',
 '3350',
 'peachtree',
 'road',
 'ne',
 'suite',
 '150atlanta',
 'georgia',
 '30326',
 'address',
 'principal',
 'executive',
 'offices',
 'zip',
 'code',
 'registrant',
 'telephone',
 'number',
 'include',
 'a

In [32]:
###take out strings containing numbers

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def clean_numb(text):
    
    text = [item for subitem in text for item in subitem.split() if item.isdigit()==False]  
    text = [item for subitem in text for item in subitem.split() if has_numbers(item)==False]
    return text


for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        for ten_k in tqdm(ten_ks, desc='Cleaning {} 10-Ks'.format(ticker), unit='10-K'):
            ten_k['file_clean_numb'] = clean_numb(ten_k['file_lemma'])
    except:
        print
print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_clean_numb'])

Cleaning ODFL 10-Ks: 100%|██████████| 4/4 [00:00<00:00, 35.8210-K/s]
Cleaning OGN 10-Ks: 010-K [00:00, ?10-K/s]
Cleaning PHM 10-Ks: 100%|██████████| 4/4 [00:00<00:00, 24.8610-K/s]


[
  {
    file_clean_numb: '['k', 'ndaq', 'htm', 'k', 'ndaq', 'gaap', 'gaap',...},
  {
    file_clean_numb: '['k', 'k', 'htm', 'k', 'document', 'false', 'ndaq...},
  {
    file_clean_numb: '['k', 'k', 'htm', 'k', 'document', 'unite', 'stat...},
  {
    file_clean_numb: '['k', 'k', 'htm', 'k', 'document', 'table', 'cont...},
]


In [33]:
ten_ks_by_ticker[example_ticker][0]['file_clean_numb']

['k',
 'ndaq',
 'htm',
 'k',
 'ndaq',
 'gaap',
 'gaap',
 'otherliabilitiescurrentus',
 'gaap',
 'commonstock',
 'usdxbrli',
 'marketservicesmemberus',
 'gaap',
 'marketservicesmemberus',
 'gaap',
 'marketservicesmemberus',
 'gaap',
 'gaap',
 'operatingsegmentsmemberndaq',
 'gaap',
 'operatingsegmentsmemberndaq',
 'gaap',
 'operatingsegmentsmemberndaq',
 'gaap',
 'operatingsegmentsmemberndaq',
 'gaap',
 'operatingsegmentsmemberndaq',
 'gaap',
 'operatingsegmentsmemberndaq',
 'markettechnologymemberus',
 'gaap',
 'markettechnologymemberus',
 'gaap',
 'markettechnologymemberus',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'cumulativeeffectperiodofadoptionadjustmentmemberus',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 'gaap',
 '

Deal with 10Q the same way

In [34]:
#Take long time  11:58-12:13  ~15min for 50comps
raw_fillings_by_ticker_10Q = {}     
for ticker, data in sec_data_10Q.items():   ###get 10Q from dict
    try:
        raw_fillings_by_ticker_10Q[ticker] = {}   ###create empty 10Q filing
        for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
            if (file_type == '10-Q'):
                file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            
            
                raw_fillings_by_ticker_10Q[ticker][file_date] = requests.get(file_url,headers=HEADER).text
    except:
        print(ticker)
print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker_10Q[example_ticker].values()))[:1000]))

NKE


Example Document:

<SEC-DOCUMENT>0001120193-21-000027.txt : 20211104
<SEC-HEADER>0001120193-21-000027.hdr.sgml : 20211104
<ACCEPTANCE-DATETIME>20211104130229
ACCESSION NUMBER:		0001120193-21-000027
CONFORMED SUBMISSION TYPE:	10-Q
PUBLIC DOCUMENT COUNT:		124
CONFORMED PERIOD OF REPORT:	20210930
FILED AS OF DATE:		20211104
DATE AS OF CHANGE:		20211104

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			NASDAQ, INC.
		CENTRAL INDEX KEY:			0001120193
		STANDARD INDUSTRIAL CLASSIFICATION:	SECURITY & COMMODITY BROKERS, DEALERS, EXCHANGES & SERVICES [6200]
		IRS NUMBER:				521165937
		STATE OF INCORPORATION:			DE
		FISCAL YEAR END:			1231

	FILING VALUES:
		FORM TYPE:		10-Q
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	001-38855
		FILM NUMBER:		211379149

	BUSINESS ADDRESS:	
		STREET 1:		151 W. 42ND STREET
		CITY:			NEW YORK
		STATE:			NY
		ZIP:			10036
		BUSINESS PHONE:		1 212 401 8700

	MAIL ADDRESS:	
		STREET 1:		151 W. 42ND STREET
		CITY:			NEW YORK
		STATE:			NY
		ZIP:			10036

	FORMER COMPAN

In [35]:
filling_documents_by_ticker_10Q = {}
for ticker, raw_fillings in raw_fillings_by_ticker_10Q.items():
    try:
        filling_documents_by_ticker_10Q[ticker] = {}
        for file_date, filling in tqdm(raw_fillings.items(), desc='Getting Documents from {} Fillings'.format(ticker), unit='filling'):
            filling_documents_by_ticker_10Q[ticker][file_date] = get_documents(filling)
    except:
        print(ticker)
print('\n\n'.join([
    'Document {} Filed on {}:\n{}...'.format(doc_i, file_date, doc[:200])
    for file_date, docs in filling_documents_by_ticker_10Q[example_ticker].items()
    for doc_i, doc in enumerate(docs)][:3]))

Getting Documents from PHM Fillings: 100%|██████████| 15/15 [00:01<00:00, 10.64filling/s]

Document 0 Filed on 2021-11-04:

<TYPE>10-Q
<SEQUENCE>1
<FILENAME>ndaq-20210930.htm
<DESCRIPTION>10-Q
<TEXT>
<XBRL>
<?xml version="1.0" ?><!--XBRL Document Created with Wdesk from Workiva--><!--Copyright 2021 Workiva--><!--r:d31cd6a...

Document 1 Filed on 2021-11-04:

<TYPE>EX-31.1
<SEQUENCE>2
<FILENAME>ndaq9302021ex-311.htm
<DESCRIPTION>EX-31.1
<TEXT>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head...

Document 2 Filed on 2021-11-04:

<TYPE>EX-31.2
<SEQUENCE>3
<FILENAME>ndaq9302021ex-312.htm
<DESCRIPTION>EX-31.2
<TEXT>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head...


In [36]:
#1 min

ten_Qs_by_ticker = {}
for ticker, filling_documents in filling_documents_by_ticker_10Q.items():
    ten_Qs_by_ticker[ticker] = []
    for file_date, documents in filling_documents.items():
        for document in documents:
            if get_document_type(document) == '10-q':
                ten_Qs_by_ticker[ticker].append({
                    'cik': cik_lookup[ticker],
                    'file': document,
                    'file_date': file_date})
print_ten_k_data(ten_Qs_by_ticker[example_ticker][:5], ['cik', 'file', 'file_date'])    ###use print_ten_k_data to print 10Q

[
  {
    cik: '1120193'
    file: '\n<TYPE>10-Q\n<SEQUENCE>1\n<FILENAME>ndaq-2021093...
    file_date: '2021-11-04'},
  {
    cik: '1120193'
    file: '\n<TYPE>10-Q\n<SEQUENCE>1\n<FILENAME>ndaq-2021063...
    file_date: '2021-08-04'},
  {
    cik: '1120193'
    file: '\n<TYPE>10-Q\n<SEQUENCE>1\n<FILENAME>ndaq-2021033...
    file_date: '2021-05-05'},
  {
    cik: '1120193'
    file: '\n<TYPE>10-Q\n<SEQUENCE>1\n<FILENAME>ndaq-2020093...
    file_date: '2020-11-04'},
  {
    cik: '1120193'
    file: '\n<TYPE>10-Q\n<SEQUENCE>1\n<FILENAME>ndaq-2020063...
    file_date: '2020-08-05'},
]


In [37]:

#Take long time   1:35pm-1:58  20min or 50comps

##slice_tenQ=list(ten_Qs_by_ticker.keys())
##ten_Qs_by_ticker_sliced= {key: ten_Qs_by_ticker[key] for key in slice_tenQ[:9] }    #ADBE is indexed-9

for ticker, ten_Qs in ten_Qs_by_ticker.items():
    try:
        for ten_Q in tqdm(ten_Qs, desc='Cleaning {} 10-Qs'.format(ticker), unit='10-Q'):
            ten_Q['file_clean'] = clean_text(ten_Q['file'])
    except:
        print(ticker)
        #del ten_Qs_by_ticker[ticker]
        
print_ten_k_data(ten_Qs_by_ticker[example_ticker][:5], ['file_clean'])

#enountered error at ADBE, so skipped ADBE's 10Q 
#next time encounter error just skip and print tickers

Cleaning PHM 10-Qs: 100%|██████████| 15/15 [00:31<00:00,  2.09s/10-Q]

[
  {
    file_clean: '10-q\n1\nndaq-20210930.htm\n10-q\n\n\n\nndaq-2021...},
  {
    file_clean: '10-q\n1\nndaq-20210630.htm\n10-q\n\n\n\nndaq-2021...},
  {
    file_clean: '10-q\n1\nndaq-20210331.htm\n10-q\n\n\n\nndaq-2021...},
  {
    file_clean: '10-q\n1\nndaq-20200930.htm\n10-q\n\n\n\nndaq-2020...},
  {
    file_clean: '10-q\n1\nndaq-20200630.htm\n10-q\n\n\n\nndaq-2020...},
]


In [38]:
ten_Qs_by_ticker.keys()

dict_keys(['NDAQ', 'NEE', 'NEM', 'NFLX', 'NFX', 'NI', 'NKE', 'NKTR', 'NLOK', 'NLSN', 'NOC', 'NOV', 'NOW', 'NRG', 'NSC', 'NTAP', 'NTRS', 'NUE', 'NVDA', 'NVR', 'NWL', 'NWSA', 'NXPI', 'O', 'ODFL', 'OGN', 'OKE', 'OMC', 'ORCL', 'ORLY', 'OTIS', 'OXY', 'PAYC', 'PAYX', 'PBCT', 'PBI', 'PCAR', 'PCG', 'PCLN', 'PDCO', 'PEAK', 'PEG', 'PENN', 'PEP', 'PFE', 'PFG', 'PG', 'PGR', 'PH', 'PHM'])

In [39]:
word_pattern = re.compile('\w+')
for ticker, ten_Qs in ten_Qs_by_ticker.items():
    try:
        for ten_Q in tqdm(ten_Qs, desc='Lemmatize {} 10-Qs'.format(ticker), unit='10-Q'):
            ten_Q['file_lemma'] = lemmatize_words(word_pattern.findall(ten_Q['file_clean']))
    except:
        print(ticker)
print_ten_k_data(ten_Qs_by_ticker[example_ticker][:5], ['file_lemma'])

Lemmatize PHM 10-Qs: 100%|██████████| 15/15 [00:01<00:00,  9.1310-Q/s]

[
  {
    file_lemma: '['10', 'q', '1', 'ndaq', '20210930', 'htm', '10',...},
  {
    file_lemma: '['10', 'q', '1', 'ndaq', '20210630', 'htm', '10',...},
  {
    file_lemma: '['10', 'q', '1', 'ndaq', '20210331', 'htm', '10',...},
  {
    file_lemma: '['10', 'q', '1', 'ndaq', '20200930', 'htm', '10',...},
  {
    file_lemma: '['10', 'q', '1', 'ndaq', '20200630', 'htm', '10',...},
]


In [40]:
from nltk.corpus import stopwords
lemma_english_stopwords = lemmatize_words(stopwords.words('english'))
for ticker, ten_Qs in ten_Qs_by_ticker.items():
    try:
        for ten_Q in tqdm(ten_Qs, desc='Remove Stop Words for {} 10-Qs'.format(ticker), unit='10-Q'):
            ten_Q['file_lemma'] = [word for word in ten_Q['file_lemma'] if word not in lemma_english_stopwords]
    except:
        print(ticker)
print('Stop Words Removed')

Remove Stop Words for PHM 10-Qs: 100%|██████████| 15/15 [00:00<00:00, 20.5410-Q/s]

Stop Words Removed


In [41]:
def clean_numbQ(text):
    
    text = [item for subitem in text for item in subitem.split() if item.isdigit()==False]  
    text = [item for subitem in text for item in subitem.split() if has_numbers(item)==False]
    return text

for ticker, ten_Qs in ten_Qs_by_ticker.items():
    try:
        for ten_Q in tqdm(ten_Qs, desc='Cleaning {} 10-Qs'.format(ticker), unit='10-Q'):
            ten_Q['file_clean_numb'] = clean_numbQ(ten_Q['file_lemma'])
    except:
        print(ticker)
print_ten_k_data(ten_Qs_by_ticker[example_ticker][:5], ['file_clean_numb'])

Cleaning PHM 10-Qs: 100%|██████████| 15/15 [00:00<00:00, 50.8510-Q/s]

[
  {
    file_clean_numb: '['q', 'ndaq', 'htm', 'q', 'ndaq', 'fasb', 'org', ...},
  {
    file_clean_numb: '['q', 'ndaq', 'htm', 'q', 'ndaq', 'fasb', 'org', ...},
  {
    file_clean_numb: '['q', 'ndaq', 'htm', 'q', 'ndaq', 'gaap', 'gaap',...},
  {
    file_clean_numb: '['q', 'ndaq', 'htm', 'q', 'ndaq', 'gaap', 'gaap',...},
  {
    file_clean_numb: '['q', 'ndaq', 'htm', 'q', 'ndaq', 'gaap', 'gaap',...},
]


In [42]:
sentiments = ['negative']

Fin_Neg=pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
#Fin_Neg=Fin_Neg.loc[Fin_Neg['Negative']>=2009]


sentiment_df = Fin_Neg
sentiment_df.columns = [column.lower() for column in sentiment_df.columns] # Lowercase the columns for ease of use

# Remove unused information
sentiment_df = sentiment_df[sentiments + ['word']]
sentiment_df[sentiments] = sentiment_df[sentiments].astype(bool)
sentiment_df = sentiment_df[(sentiment_df[sentiments]).any(1)]

# Apply the same preprocessing to these words as the 10-k words
sentiment_df['word'] = lemmatize_words(sentiment_df['word'].str.lower())
sentiment_df = sentiment_df.drop_duplicates('word')


sentiment_df.head()

/var/folders/d4/k_xt3lys2znb_23f5xrq4y600000gn/T/ipykernel_17704/1202980683.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_df[sentiments] = sentiment_df[sentiments].astype(bool)
/var/folders/d4/k_xt3lys2znb_23f5xrq4y600000gn/T/ipykernel_17704/1202980683.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_df['word'] = lemmatize_words(sentiment_df['word'].str.lower())


,negative,word
9,True,abandon
12,True,abandonment
13,True,abandonments
51,True,abdicate
54,True,abdication


In [43]:
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
#import project_helper
def get_bag_of_words(sentiment_words, docs):

    vec = CountVectorizer(vocabulary=sentiment_words)
    vectors = vec.fit_transform(docs)
    words_list = vec.get_feature_names()
    bag_of_words = np.zeros([len(docs), len(words_list)])
    
    for i in range(len(docs)):
        bag_of_words[i] = vectors[i].toarray()[0]
    return bag_of_words.astype(int)

In [44]:
sentiment_bow_ten_ks = {}   #####setup ONCE?

for ticker, ten_ks in ten_ks_by_ticker.items():
    lemma_docs = [' '.join(ten_k['file_clean_numb']) for ten_k in ten_ks]

    sentiment_bow_ten_ks[ticker] = {sentiment: get_bag_of_words(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
       for sentiment in sentiments}


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [45]:
def get_negative_word_countK(ticker):
    return np.cumsum(sentiment_bow_ten_ks[ticker]['negative'], axis=1)[:,-1]


def get_total_word_countK(ticker):
    total=[0]*len(ten_ks_by_ticker[ticker])
    for i in range(len(ten_ks_by_ticker[ticker])):
        total[i]=len(ten_ks_by_ticker[ticker][i]['file_clean_numb'])
    return total   

In [46]:
FNproporK={}
for ticker in ten_ks_by_ticker.keys():
    FNproporK[ticker]=get_negative_word_countK(ticker)/get_total_word_countK(ticker)

In [47]:
def get_file_dateK(ticker):
    fdate=[0]*len(ten_ks_by_ticker[ticker])
    for i in range(len(ten_ks_by_ticker[ticker])):
        fdate[i]=ten_ks_by_ticker[ticker][i]['file_date']
    return fdate

fdatek={}
for ticker in ten_ks_by_ticker.keys():
    fdatek[ticker]=get_file_dateK(ticker)

In [48]:
def get_FNproporK_date(fdatek,proporK):
    FNproporKDATE={}
    for ticker in fdatek.keys():
        FNproporK_date= {fdatek[ticker][i]: proporK[ticker][i] for i in range(len(proporK[ticker]))}
        FNproporKDATE.update(FNproporK_date)
    return FNproporKDATE

In [49]:
FN_10K_portion=get_FNproporK_date(fdatek,FNproporK)    ###date: proportion of 10K negaword FinNeg
#AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAaaaaaaaaaa

In [50]:
colname=['Word'] 
H4N=pd.read_csv('Harvard IV_Negative Word List_Inf.txt',names=colname,header=None)

H4N['negative']=True
#H4N

In [51]:
sentiments = ['negative']

#Fin_Neg=pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
#Fin_Neg=Fin_Neg.loc[Fin_Neg['Negative']>=2009]


Hsentiment_df = H4N
Hsentiment_df.columns = [column.lower() for column in Hsentiment_df.columns] # Lowercase the columns for ease of use

# Remove unused information
Hsentiment_df = Hsentiment_df[sentiments + ['word']]
Hsentiment_df[sentiments] = Hsentiment_df[sentiments].astype(bool)
Hsentiment_df = Hsentiment_df[(Hsentiment_df[sentiments]).any(1)]

# Apply the same preprocessing to these words as the 10-k words
Hsentiment_df['word'] = lemmatize_words(Hsentiment_df['word'].str.lower())
Hsentiment_df = Hsentiment_df.drop_duplicates('word')


Hsentiment_df.head()

,negative,word
0,True,abandon
3,True,abandonment
4,True,abandonments
6,True,abate
8,True,abatement


In [52]:
Hsentiment_bow_ten_ks = {}   #####setup ONCE?

for ticker, ten_ks in ten_ks_by_ticker.items():
    lemma_docs = [' '.join(ten_k['file_clean_numb']) for ten_k in ten_ks]

    Hsentiment_bow_ten_ks[ticker] = {sentiment: get_bag_of_words(Hsentiment_df[Hsentiment_df[sentiment]]['word'], lemma_docs)
       for sentiment in sentiments}

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [53]:
def Hget_negative_word_countK(ticker):
    return np.cumsum(Hsentiment_bow_ten_ks[ticker]['negative'], axis=1)[:,-1]


HproporK={}
for ticker in ten_ks_by_ticker.keys():
    HproporK[ticker]=Hget_negative_word_countK(ticker)/get_total_word_countK(ticker)

In [54]:
def get_HproporK_date(fdatek,HproporK):
    HproporKDATE={}
    for ticker in fdatek.keys():
        HproporK_date= {fdatek[ticker][i]: HproporK[ticker][i] for i in range(len(HproporK[ticker]))}
        HproporKDATE.update(HproporK_date)
    return HproporKDATE

In [55]:
H_10K_portion=get_HproporK_date(fdatek,HproporK) 

In [56]:
def get_file_dateQ(ticker):
    fdate=[0]*len(ten_Qs_by_ticker[ticker])
    for i in range(len(ten_Qs_by_ticker[ticker])):
        fdate[i]=ten_Qs_by_ticker[ticker][i]['file_date']
    return fdate

fdateQ={}
for ticker in ten_Qs_by_ticker.keys():
    fdateQ[ticker]=get_file_dateQ(ticker)

In [57]:
sentiment_bow_ten_Qs = {}   #####setup ONCE?

for ticker, ten_Qs in ten_Qs_by_ticker.items():
    lemma_docs = [' '.join(ten_Q['file_clean_numb']) for ten_Q in ten_Qs]

    sentiment_bow_ten_Qs[ticker] = {sentiment: get_bag_of_words(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
       for sentiment in sentiments}

print_ten_k_data([sentiment_bow_ten_Qs[example_ticker]], sentiments)

[
  {
    negative: '[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ....},
]


In [58]:
def get_negative_word_countQ(ticker):
    return np.cumsum(sentiment_bow_ten_Qs[ticker]['negative'], axis=1)[:,-1]

def get_total_word_countQ(ticker):
    total=[0]*len(ten_Qs_by_ticker[ticker])
    for i in range(len(ten_Qs_by_ticker[ticker])):
        total[i]=len(ten_Qs_by_ticker[ticker][i]['file_clean_numb'])
    return total   

In [59]:
FNproporQ={}
for ticker in ten_Qs_by_ticker.keys():
    FNproporQ[ticker]=get_negative_word_countQ(ticker)/get_total_word_countQ(ticker)
#get_total_word_count('A')
len(FNproporQ)

50

In [60]:
def get_file_dateQ(ticker):
    fdate=[0]*len(ten_Qs_by_ticker[ticker])
    for i in range(len(ten_Qs_by_ticker[ticker])):
        fdate[i]=ten_Qs_by_ticker[ticker][i]['file_date']
    return fdate

fdateQ={}
for ticker in ten_Qs_by_ticker.keys():
    fdateQ[ticker]=get_file_dateQ(ticker)

In [61]:
def get_FNproporQ_date(fdateQ,proporQ):
    FNproporQDATE={}
    for ticker in fdateQ.keys():
        FNproporQ_date= {fdateQ[ticker][i]: proporQ[ticker][i] for i in range(len(proporQ[ticker]))}
        FNproporQDATE.update(FNproporQ_date)
    return FNproporQDATE

In [62]:
FN_10Q_portion=get_FNproporQ_date(fdateQ,FNproporQ)  

In [63]:
Hsentiment_bow_ten_Qs = {}   #####setup ONCE?

for ticker, ten_Qs in ten_Qs_by_ticker.items():
    lemma_docs = [' '.join(ten_Q['file_clean_numb']) for ten_Q in ten_Qs]

    Hsentiment_bow_ten_Qs[ticker] = {sentiment: get_bag_of_words(Hsentiment_df[Hsentiment_df[sentiment]]['word'], lemma_docs)
       for sentiment in sentiments}

print_ten_k_data([Hsentiment_bow_ten_Qs[example_ticker]], sentiments)

[
  {
    negative: '[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ....},
]


In [64]:
def Hget_negative_word_countQ(ticker):
    return np.cumsum(Hsentiment_bow_ten_Qs[ticker]['negative'], axis=1)[:,-1]

HproporQ={}
for ticker in ten_Qs_by_ticker.keys():
    HproporQ[ticker]=Hget_negative_word_countQ(ticker)/get_total_word_countQ(ticker)
    
def get_HproporQ_date(fdateQ,HproporQ):
    HproporQDATE={}
    for ticker in fdateQ.keys():
        HproporQ_date= {fdateQ[ticker][i]: HproporQ[ticker][i] for i in range(len(HproporQ[ticker]))}
        HproporQDATE.update(HproporQ_date)
    return HproporQDATE

In [65]:
H_10Q_portion=get_HproporQ_date(fdateQ,HproporQ)
H_10Q_portion

{'2021-11-04': 0.05223381831268315,
 '2021-08-04': 0.0642247369465952,
 '2021-05-05': 0.073108425918417,
 '2020-11-04': 0.05560854635635254,
 '2020-08-05': 0.06314141589841711,
 '2020-05-06': 0.06604292790313704,
 '2019-11-06': 0.06282867423194022,
 '2019-08-05': 0.07621428902162337,
 '2019-05-01': 0.06002002423986932,
 '2018-11-06': 0.087574486868241,
 '2018-08-01': 0.09247981112333324,
 '2018-05-02': 0.07411012235817575,
 '2017-11-07': 0.09208103130755065,
 '2017-08-02': 0.07965189663190061,
 '2017-05-10': 0.0742989750531812,
 '2021-10-25': 0.0641555414282687,
 '2021-07-26': 0.06328562877509931,
 '2021-04-23': 0.0631730942783226,
 '2020-10-23': 0.061265216523647976,
 '2020-07-24': 0.06215731758751582,
 '2020-04-23': 0.07755348516218082,
 '2019-10-23': 0.07421576128538639,
 '2019-07-24': 0.06939040207522698,
 '2019-04-23': 0.07352011796924374,
 '2018-10-23': 0.07106120349734271,
 '2018-07-25': 0.07792647806200237,
 '2018-04-24': 0.07448121144139092,
 '2017-10-26': 0.05529773187790052,

In [66]:
###TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer
def get_tfidf(sentiment_words, docs):
    
    vec = TfidfVectorizer(vocabulary=sentiment_words)
    tfidf = vec.fit_transform(docs)
    
    return tfidf.toarray()

In [67]:
##10K tfidf   FN

FNsentiment_tfidf_ten_ks = {}
for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        lemma_docs = [' '.join(ten_k['file_lemma']) for ten_k in ten_ks]
    
        FNsentiment_tfidf_ten_ks[ticker] = {
            sentiment: get_tfidf(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
            for sentiment in sentiments}
        
    except:
        print(ticker)
print_ten_k_data([FNsentiment_tfidf_ten_ks[example_ticker]], sentiments)

OGN
[
  {
    negative: '[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0. 0....},
]


In [70]:
FNsentiment_tfidf_ten_ks['OGN']={'negative':np.empty([5,1522])}


In [71]:
FN_tfidf_K={}
for ticker in ten_ks_by_ticker.keys():
    array=np.cumsum(FNsentiment_tfidf_ten_ks[ticker]['negative'], axis=1)[:,-1]
    FN_tfidf_K[ticker]=array
#FN_tfidf_K                                       ####### FN tfidf for 10K 

In [72]:
FN_tfidf_K_DATE={}
for ticker in fdatek.keys():
    try:
        FNproporK_date= {fdatek[ticker][i]: FN_tfidf_K[ticker][i] for i in range(len(FN_tfidf_K[ticker]))}
        FN_tfidf_K_DATE.update(FNproporK_date)
    except:
        # FNproporK_date=
        print(ticker)
FN_tfidf_K_DATE   

OGN


{'2021-02-23': 5.670157884213835,
 '2020-02-25': 6.119409730149488,
 '2019-02-22': 4.0445004147644985,
 '2018-02-28': 6.4993968822173365,
 '2021-02-12': 4.726890376325529,
 '2020-02-14': 4.559125832273654,
 '2019-02-15': 6.710456425528052,
 '2018-02-16': 5.279233499993857,
 '2022-02-24': 5.863947058826318,
 '2021-02-18': 7.658471186974129,
 '2020-02-20': 5.428313846969509,
 '2019-02-21': 4.089340252555157,
 '2021-01-28': 8.097742895196353,
 '2020-01-29': 6.573996505194672,
 '2019-01-29': 6.2621223571642375,
 '2018-02-20': 5.313710815830532,
 '2017-02-21': 6.839227780216095,
 '2015-02-24': 6.166907081627869,
 '2021-02-17': 4.198114453520416,
 '2020-02-28': 6.562419675166499,
 '2019-02-20': 5.427733118907483,
 '2021-07-20': 6.989185800169403,
 '2020-07-24': 6.88803880254472,
 '2019-07-23': 6.138567531333913,
 '2018-07-25': 6.591154327561645,
 '2021-02-26': 5.4362060870705955,
 '2019-03-01': 7.297143883600912,
 '2018-03-01': 5.681742191937446,
 '2021-05-21': 7.454452370610131,
 '2020-05-2

In [73]:
##10K tfidf   H4N

Hsentiment_tfidf_ten_ks = {}
for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        lemma_docs = [' '.join(ten_k['file_lemma']) for ten_k in ten_ks]
    
        Hsentiment_tfidf_ten_ks[ticker] = {
            sentiment: get_tfidf(Hsentiment_df[Hsentiment_df[sentiment]]['word'], lemma_docs)
            for sentiment in sentiments}
    except:
        print(ticker)
#print_ten_k_data([FNsentiment_tfidf_ten_ks[example_ticker]], sentiments)


H_tfidf_K={}
for ticker in ten_ks_by_ticker.keys():
    try:
        array=np.cumsum(Hsentiment_tfidf_ten_ks[ticker]['negative'], axis=1)[:,-1]
        H_tfidf_K[ticker]=array
    except:
        print(ticker)
#H_tfidf_K         


H_tfidf_K_DATE={}
for ticker in fdatek.keys():
    try:
        HproporK_date= {fdatek[ticker][i]: H_tfidf_K[ticker][i] for i in range(len(H_tfidf_K[ticker]))}
        H_tfidf_K_DATE.update(HproporK_date)
    except:
        print(ticker)
H_tfidf_K_DATE 

OGN
OGN
OGN


{'2021-02-23': 6.108127847235397,
 '2020-02-25': 5.905120601873412,
 '2019-02-22': 3.717683221699154,
 '2018-02-28': 5.3284317619380595,
 '2021-02-12': 5.5527739493340365,
 '2020-02-14': 5.447766499542225,
 '2019-02-15': 5.595863109605552,
 '2018-02-16': 6.186186102992463,
 '2022-02-24': 4.794774514618774,
 '2021-02-18': 5.886758800503371,
 '2020-02-20': 5.074548595730279,
 '2019-02-21': 4.778391273402677,
 '2021-01-28': 5.180932159093467,
 '2020-01-29': 5.072052584034844,
 '2019-01-29': 4.581475048685479,
 '2018-02-20': 4.818986918675318,
 '2017-02-21': 5.807447567031803,
 '2015-02-24': 5.825843704995864,
 '2021-02-17': 5.144490792599641,
 '2020-02-28': 5.245231085083622,
 '2019-02-20': 4.894337533472149,
 '2021-07-20': 5.732802988396436,
 '2020-07-24': 5.744080041969609,
 '2019-07-23': 5.305195686291684,
 '2018-07-25': 5.268208086021383,
 '2021-02-26': 6.186490796688145,
 '2019-03-01': 6.694510735661068,
 '2018-03-01': 5.474692923296058,
 '2021-05-21': 6.056275750873289,
 '2020-05-28

In [74]:
def get_file_dateQ(ticker):
    fdate=[0]*len(ten_Qs_by_ticker[ticker])
    for i in range(len(ten_Qs_by_ticker[ticker])):
        fdate[i]=ten_Qs_by_ticker[ticker][i]['file_date']
    return fdate

fdateQ={}
for ticker in ten_Qs_by_ticker.keys():
    fdateQ[ticker]=get_file_dateQ(ticker)

In [75]:
##10Q tfidf   FN

FNsentiment_tfidf_ten_Qs = {}
for ticker, ten_Qs in ten_Qs_by_ticker.items():
    lemma_docs = [' '.join(ten_Q['file_lemma']) for ten_Q in ten_Qs]
    
    FNsentiment_tfidf_ten_Qs[ticker] = {
        sentiment: get_tfidf(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
        for sentiment in sentiments}
#print_ten_k_data([FNsentiment_tfidf_ten_Qs[example_ticker]], sentiments)


FN_tfidf_Q={}
for ticker in ten_Qs_by_ticker.keys():
    array=np.cumsum(FNsentiment_tfidf_ten_Qs[ticker]['negative'], axis=1)[:,-1]
    FN_tfidf_Q[ticker]=array
#FN_tfidf_Q                                       ####### FN tfidf for 10Q 

FN_tfidf_Q_DATE={}
for ticker in fdateQ.keys():
    FNproporQ_date= {fdateQ[ticker][i]: FN_tfidf_Q[ticker][i] for i in range(len(FN_tfidf_Q[ticker]))}
    FN_tfidf_Q_DATE.update(FNproporQ_date)
FN_tfidf_Q_DATE                                                      ####### FN tfidf for 10Q 

{'2021-11-04': 4.160069590570587,
 '2021-08-04': 2.4780941217494563,
 '2021-05-05': 4.401152888629155,
 '2020-11-04': 4.5635776087089654,
 '2020-08-05': 4.537605189332772,
 '2020-05-06': 4.925213165122985,
 '2019-11-06': 3.47789569729541,
 '2019-08-05': 4.5904010749618855,
 '2019-05-01': 3.3138858435419323,
 '2018-11-06': 4.357946383226145,
 '2018-08-01': 3.2404385733398255,
 '2018-05-02': 3.6239151123083775,
 '2017-11-07': 7.0756983051243925,
 '2017-08-02': 3.1430424381916127,
 '2017-05-10': 3.08874230802242,
 '2021-10-25': 3.2910992963847656,
 '2021-07-26': 3.286321932386762,
 '2021-04-23': 3.6853716332030206,
 '2020-10-23': 3.6481230542593748,
 '2020-07-24': 3.5008404924330576,
 '2020-04-23': 5.1876198811647365,
 '2019-10-23': 4.114870190668261,
 '2019-07-24': 3.9660811963943123,
 '2019-04-23': 3.8581213598990196,
 '2018-10-23': 3.5250368756964456,
 '2018-07-25': 4.867433079150718,
 '2018-04-24': 3.555612082713208,
 '2017-10-26': 4.121897061699879,
 '2017-07-26': 4.194831945488509,


In [76]:
##10Q tfidf   H

Hsentiment_tfidf_ten_Qs = {}
for ticker, ten_Qs in ten_Qs_by_ticker.items():
    lemma_docs = [' '.join(ten_Q['file_lemma']) for ten_Q in ten_Qs]
    
    Hsentiment_tfidf_ten_Qs[ticker] = {
        sentiment: get_tfidf(Hsentiment_df[Hsentiment_df[sentiment]]['word'], lemma_docs)
        for sentiment in sentiments}
#print_ten_k_data([Hsentiment_tfidf_ten_Qs[example_ticker]], sentiments)


H_tfidf_Q={}
for ticker in ten_Qs_by_ticker.keys():
    array=np.cumsum(Hsentiment_tfidf_ten_Qs[ticker]['negative'], axis=1)[:,-1]
    H_tfidf_Q[ticker]=array
#FN_tfidf_Q                                       ####### H tfidf for 10Q 

H_tfidf_Q_DATE={}
for ticker in fdateQ.keys():
    HproporQ_date= {fdateQ[ticker][i]: H_tfidf_Q[ticker][i] for i in range(len(H_tfidf_Q[ticker]))}
    H_tfidf_Q_DATE.update(HproporQ_date)
H_tfidf_Q_DATE                                                      ####### H tfidf for 10Q 

{'2021-11-04': 4.861009276867934,
 '2021-08-04': 4.07048356193724,
 '2021-05-05': 5.3526105144086245,
 '2020-11-04': 5.521121394774188,
 '2020-08-05': 5.701333164207557,
 '2020-05-06': 5.50348444985712,
 '2019-11-06': 4.638119887761338,
 '2019-08-05': 4.855941456389041,
 '2019-05-01': 4.712349608038763,
 '2018-11-06': 4.603639858613248,
 '2018-08-01': 4.99343577038063,
 '2018-05-02': 4.519271324461196,
 '2017-11-07': 5.193195323442642,
 '2017-08-02': 4.610837189754058,
 '2017-05-10': 4.994273155560191,
 '2021-10-25': 5.36495013298771,
 '2021-07-26': 5.336747605277775,
 '2021-04-23': 5.45122646628416,
 '2020-10-23': 5.432302240981102,
 '2020-07-24': 5.243350343536827,
 '2020-04-23': 5.8509321096977365,
 '2019-10-23': 5.520284117365945,
 '2019-07-24': 5.410821692003877,
 '2019-04-23': 5.097739915554905,
 '2018-10-23': 4.813501258534134,
 '2018-07-25': 5.61773040464051,
 '2018-04-24': 4.684773355088073,
 '2017-10-26': 4.86256149708792,
 '2017-07-26': 5.481610735355415,
 '2017-04-21': 5.31

In [77]:
c10k= [H_10K_portion, FN_10K_portion,H_tfidf_K_DATE,FN_tfidf_K_DATE]
K={}

for k in c10k[0].keys():
  K[k] = list(d[k] for d in c10k)

In [78]:

c10Q= [H_10Q_portion,FN_10Q_portion,H_tfidf_Q_DATE,FN_tfidf_Q_DATE]
Q={}

for k in c10Q[0].keys():
  Q[k] = list(d[k] for d in c10Q)

In [79]:
dfK = pd.DataFrame.from_dict(K).T
dfQ=pd.DataFrame.from_dict(Q).T


In [80]:
frames = [dfK,dfQ]

DF0_50= pd.concat(frames)
DF0_50.columns = ['H4N_portion','FN_portion','H_tfidf','FN_tfidf']
DF0_50

,H4N_portion,FN_portion,H_tfidf,FN_tfidf
2021-02-23,0.075985,0.026614,6.108128,5.670158
2020-02-25,0.067435,0.022404,5.905121,6.119410
2019-02-22,0.073157,0.025132,3.717683,4.044500
2018-02-28,0.068405,0.024508,5.328432,6.499397
2021-02-12,0.085244,0.031715,5.552774,4.726890
...,...,...,...,...
2018-02-08,0.076081,0.017207,4.627231,3.579735
2017-02-07,0.072905,0.018192,4.863861,3.922112
2020-07-23,0.077608,0.033581,5.789979,4.574950
2019-07-23,0.072366,0.027958,4.881956,3.650253


In [81]:
DF0_50.to_csv('DF_401_450.csv')

In [ ]:
DF0_50